<a href="https://colab.research.google.com/github/SLy-Precision/Player-Selection-and-Optimization-for-Roster/blob/main/00_Preprocess_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis and preprocess data from statsbomb

# - From MAPLIS FC

## First part: Analysis of APIS and data from statsbomb

In [ ]:
# Statsbomb api
!pip install --quiet statsbombpy
!pip install --quiet mplsoccer
!pip install --quiet highlight_text
!pip install --quiet mplsoccer
!pip install --quiet highlight_text

# Libraries for the data manipulation and plot
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch, Pitch, Sbapi
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.4/86.4 kB 2.1 MB/s eta 0:00:00


In [ ]:
#Credentials for Statsbomb from ITAM
user = "itam_hackathon@hudl.com"
password = "pGwIprel"

# Get all the competitions available in Statsbomb
competitions = sb.competitions(creds={"user": user, "passwd": password})

# We print the competitions to see which ones are available
competitions


,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,73,317,Mexico,Liga MX,male,False,False,2024/2025,2025-08-30T16:14:20.970616,2025-08-30T16:17:12.621755,2025-08-30T16:17:12.621755,2025-08-30T16:14:20.970616
1,73,281,Mexico,Liga MX,male,False,False,2023/2024,2024-12-20T23:40:31.103974,2025-06-22T23:05:32.482369,2025-06-22T23:05:32.482369,2024-12-20T23:40:31.103974
2,73,235,Mexico,Liga MX,male,False,False,2022/2023,2024-09-28T11:05:11.667984,2025-09-07T08:28:48.058191,2025-09-07T08:28:48.058191,2024-09-28T11:05:11.667984
3,73,108,Mexico,Liga MX,male,False,False,2021/2022,2024-12-10T08:59:57.612449,2024-12-10T09:06:01.294195,2024-12-10T09:06:01.294195,2024-12-10T08:59:57.612449


In [ ]:
# Arrays for competition_id and season_id
competition_id = [73]
season_id = [108,235,281,317]

# Let's print the 2021-2022 season matches for example
matches = sb.matches(competition_id=73, season_id=108, creds={"user": user, "passwd": password})

matches.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,attendance,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3816809,2022-04-16,04:06:00.000,Mexico - Liga MX,2021/2022,Tijuana,América,1,3,None,...,2024-10-14T16:57:17.228968,14,Clausura,Estadio Caliente,Luis Enrique Santander Aguirre,Sebastián Ariel Méndez Pardiñas,Fernando Ortiz,1.1.0,2,2
1,3816796,2022-01-09,05:00:00.000,Mexico - Liga MX,2021/2022,Cruz Azul,Tijuana,2,0,None,...,2024-10-14T17:00:05.129048,1,Clausura,Estadio Azteca,Daniel Quintero Huittrón,Juan Máximo Reynoso Guzmán,Sebastián Ariel Méndez Pardiñas,1.1.0,2,2
2,3816740,2022-02-12,03:00:00.000,Mexico - Liga MX,2021/2022,Mazatlán,Tijuana,2,0,None,...,2024-10-14T16:58:59.426783,5,Clausura,Estadio El Encanto,Fernando Guerrero Ramírez,Beñat San José Gil,Sebastián Ariel Méndez Pardiñas,1.1.0,2,2
3,3816789,2022-04-12,04:00:00.000,Mexico - Liga MX,2021/2022,Pachuca,Tijuana,0,0,None,...,2024-10-14T16:57:46.969055,13,Clausura,Estadio Hidalgo,Daniel Quintero Huittrón,Jorge Guillermo Almada Álves,Sebastián Ariel Méndez Pardiñas,1.1.0,2,2
4,3816723,2022-01-13,04:06:00.000,Mexico - Liga MX,2021/2022,Santos Laguna,Tigres UANL,1,1,None,...,2024-09-29T18:22:04.561656,1,Clausura,Estadio Nuevo Corona,Adonai Escobedo González,Pedro Miguel Faria Caixinha,Miguel Ernesto Herrera Aguirre,1.1.0,2,2


In [ ]:
# We will count the number of matches of every season
total_matches = 0
for i in season_id:
    matches = sb.matches(competition_id=73, season_id=i, creds={"user": user, "passwd": password})
    print(f"Season ID: {i}, Number of matches: {matches['match_id'].nunique()}")
    total_matches += matches['match_id'].nunique()

# Total
print(f"Total number of matches: {total_matches}")

Season ID: 108, Number of matches: 342
Season ID: 235, Number of matches: 342
Season ID: 281, Number of matches: 340
Season ID: 317, Number of matches: 340
Total number of matches: 1364


In [ ]:
# Let access to a dataframe of events
df_events = sb.events(match_id=3816809, creds={"user": user, "passwd": password})

df_events.head()

/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)


,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_offensive,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,...,substitution_outcome,substitution_outcome_id,substitution_replacement,substitution_replacement_id,tactics,team,team_id,timestamp,type,under_pressure
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",Tijuana,1311,00:00:00.000,Starting XI,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"{'formation': 4231, 'lineup': [{'player': {'id...",América,1229,00:00:00.000,Starting XI,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tijuana,1311,00:00:00.000,Half Start,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,América,1229,00:00:00.000,Half Start,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tijuana,1311,00:00:00.000,Half Start,NaN


In [ ]:
# We put all the columns of the df into an array
columns = df_events.columns.tolist()
columns

['ball_receipt_outcome',
 'ball_recovery_offensive',
 'ball_recovery_recovery_failure',
 'block_offensive',
 'carry_end_location',
 'clearance_aerial_won',
 'clearance_body_part',
 'clearance_head',
 'clearance_left_foot',
 'clearance_other',
 'clearance_right_foot',
 'counterpress',
 'dribble_nutmeg',
 'dribble_outcome',
 'dribble_overrun',
 'duel_outcome',
 'duel_type',
 'duration',
 'foul_committed_advantage',
 'foul_committed_card',
 'foul_committed_offensive',
 'foul_committed_penalty',
 'foul_committed_type',
 'foul_won_advantage',
 'foul_won_defensive',
 'goalkeeper_body_part',
 'goalkeeper_end_location',
 'goalkeeper_outcome',
 'goalkeeper_position',
 'goalkeeper_technique',
 'goalkeeper_type',
 'id',
 'index',
 'interception_outcome',
 'location',
 'match_id',
 'minute',
 'obv_against_after',
 'obv_against_before',
 'obv_against_net',
 'obv_for_after',
 'obv_for_before',
 'obv_for_net',
 'obv_total_net',
 'off_camera',
 'out',
 'pass_aerial_won',
 'pass_angle',
 'pass_assisted

In [ ]:
# Lets create 4 arrays with all the match_id from every season just to get the event with this

array_season_2021_2022 = sb.matches(competition_id=73, season_id=108, creds={"user": user, "passwd": password})['match_id'].to_list()
array_season_2022_2023 = sb.matches(competition_id=73, season_id=235, creds={"user": user, "passwd": password})['match_id'].to_list()
array_season_2023_2024 = sb.matches(competition_id=73, season_id=281, creds={"user": user, "passwd": password})['match_id'].to_list()
array_season_2024_2025 = sb.matches(competition_id=73, season_id=317, creds={"user": user, "passwd": password})['match_id'].to_list()

In [ ]:
# Lets check that every element in every array is unique in his array and is not in any other array
arrays = [
    array_season_2021_2022,
    array_season_2022_2023,
    array_season_2023_2024,
    array_season_2024_2025
]
names = ["2021_2022", "2022_2023", "2023_2024", "2024_2025"]

# 1) unicity
for name, arr in zip(names, arrays):
    print(name, "unique_within:", len(set(arr)) == len(arr))

# 2) check we dont have elements overlapin between them
sets = [set(a) for a in arrays]
total_items = sum(len(s) for s in sets)
union_len = len(set().union(*sets))
print("no_overlap_between_arrays:", total_items == union_len)

# If there are overlapping, which are they
for i in range(len(sets)):
    for j in range(i+1, len(sets)):
        inter = sets[i].intersection(sets[j])
        if inter:
            print(f"Overlap {names[i]} ∩ {names[j]} ->", inter)


2021_2022 unique_within: True
2022_2023 unique_within: True
2023_2024 unique_within: True
2024_2025 unique_within: True
no_overlap_between_arrays: True


## Part two: Creation of a function to get the DF we need for the algorith of Machine Learning

In [ ]:
import pandas as pd
import numpy as np
from statsbombpy import sb

# --- PRE-REQUISITE: Load your event DataFrame ---
# (Assuming you have already run this part)
# user = "itam_hackathon@hudl.com"
# password = "pGwIprel"
# creds = {"user": user, "passwd": password}
# df_events = sb.events(match_id=3816809, creds=creds)


# --- STEP 1: Get the Home Team ID (Using the correct data structure) ---
print("Obtaining match info to identify the home team...")
# Get the matches DataFrame for the relevant season
match_info_df = sb.matches(competition_id=73, season_id=108, creds=creds)
# Filter for our specific match
match_info = match_info_df[match_info_df['match_id'] == 3816809]

# Get the home team's NAME (string) from the 'matches' DataFrame.
home_team_name = match_info['home_team'].iloc[0]

# Find the first event in the 'events' DataFrame where the team name matches and get its ID.
home_team_id = df_events[df_events['team'] == home_team_name]['team_id'].iloc[0]

df_events['home_team_id'] = home_team_id
print(f"The home team is '{home_team_name}' with ID: {home_team_id}")


# --- STEP 2: Basic Preprocessing ---
print("\nStarting preprocessing...")
df = df_events.sort_values(by=['period', 'timestamp']).reset_index(drop=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S.%f', errors='coerce')
df['eventSec'] = df['timestamp'].dt.hour * 3600 + df['timestamp'].dt.minute * 60 + df['timestamp'].dt.second + df['timestamp'].dt.microsecond / 1e6


# --- STEP 3: Feature Engineering (Using Correct Column Names) ---
print("Creating the 11 variables for the model...")

# 3a. EventTypeId
event_map = {
    'Pass': 0, 'Ball Receipt': 0, 'Duel': 1, '50/50': 1,
    'Foul Committed': 2, 'Shot': 3, 'Dribble': 4, 'Carry': 5,
    'Ball Recovery': 6, 'Interception': 6, 'Block': 6, 'Clearance': 6,
    'Goal Keeper': 7, 'Dispossessed': 8, 'Miscontrol': 8, 'Pressure': 9
}
# The column is named 'type', not 'type.name'
df['EventTypeId'] = df['type'].map(event_map).fillna(10).astype(int)

# 3b. Period and Minute
df = df.rename(columns={'period': 'Period', 'minute': 'Minute'})

# 3c. Extract and Normalize Coordinates X, Y
# The column is named 'location' and contains a list [x, y]
df['x_coord'] = df['location'].apply(lambda loc: loc[0] if isinstance(loc, list) and len(loc) > 0 else np.nan)
df['y_coord'] = df['location'].apply(lambda loc: loc[1] if isinstance(loc, list) and len(loc) > 1 else np.nan)

df['X'] = (df['x_coord'] / 120) * 100
df['Y'] = (df['y_coord'] / 80) * 100
df[['X', 'Y']] = df[['X', 'Y']].fillna(50)

# 3d. IsHomeTeam
# The column is named 'team_id', not 'team.id'
df['IsHomeTeam'] = (df['team_id'].fillna(0) == df['home_team_id']).astype(int)

# 3e. IsGoal
# The column is named 'shot_outcome', not 'shot.outcome.name'
df['IsGoal'] = (df['shot_outcome'] == 'Goal').astype(int)

# 3f. IsAccurate
# We use the correct flattened column names from your list
def calculate_is_accurate_flat(row):
    if row['type'] == 'Pass' and pd.isna(row['pass_outcome']): return 1
    if row['type'] == 'Shot' and row['shot_outcome'] in ['Goal', 'Saved', 'Post']: return 1
    if row['type'] == 'Dribble' and row['dribble_outcome'] == 'Complete': return 1
    if row['type'] == 'Duel' and row['duel_outcome'] in ['Won', 'Success In Play', 'Success Out']: return 1
    return 0
df['IsAccurate'] = df.apply(calculate_is_accurate_flat, axis=1)

# 3g. HomeScore and AwayScore
df['HomeGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 1)
df['AwayGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 0)
df['HomeScore'] = df['HomeGoal'].cumsum().shift(1).fillna(0)
df['AwayScore'] = df['AwayGoal'].cumsum().shift(1).fillna(0)

# 3h. TimeDelta
df['TimeDelta'] = df['eventSec'].diff().fillna(0)


# --- STEP 4: Create the Final DataFrame ---
print("\nCreating the final DataFrame with the required columns...")
final_columns = [
    'match_id', 'eventSec', 'EventTypeId', 'Period', 'Minute', 'X', 'Y',
    'IsHomeTeam', 'IsAccurate', 'IsGoal', 'HomeScore', 'AwayScore', 'TimeDelta'
]
df_final = df[final_columns].copy().rename(columns={'match_id': 'matchId'})


# --- RESULT ---
print("\n--- Process complete! ---")
print("Preview of the DataFrame ready for training:")
print(df_final.head())
print("\nFinal DataFrame Info:")
df_final.info()

Obtaining match info to identify the home team...
The home team is 'Tijuana' with ID: 1311

Starting preprocessing...
Creating the 11 variables for the model...

Creating the final DataFrame with the required columns...

--- Process complete! ---
Preview of the DataFrame ready for training:
   matchId  eventSec  EventTypeId  Period  Minute     X     Y  IsHomeTeam  \
0  3816809     0.000           10       1       0  50.0  50.0           1   
1  3816809     0.000           10       1       0  50.0  50.0           0   
2  3816809     0.000           10       1       0  50.0  50.0           1   
3  3816809     0.000           10       1       0  50.0  50.0           0   
4  3816809     1.968            0       1       0  50.0  50.0           0   

   IsAccurate  IsGoal  HomeScore  AwayScore  TimeDelta  
0           0       0        0.0        0.0      0.000  
1           0       0        0.0        0.0      0.000  
2           0       0        0.0        0.0      0.000  
3           0    

In [ ]:
import pandas as pd
import numpy as np
from statsbombpy import sb
from tqdm import tqdm
import os

def create_full_lem_dataset_from_arrays(*season_match_id_arrays, output_filename="StatsBomb_LEM_Data.csv"):
    """
    Se conecta a la API de StatsBomb, descarga eventos para las listas de match_ids
    proporcionadas y procesa los datos al formato final para entrenar los modelos LEM.

    Args:
        *season_match_id_arrays: Un número variable de listas, donde cada lista
                                 contiene los match_ids de una temporada.
        output_filename (str): El nombre del archivo CSV donde se guardará el resultado.

    Returns:
        pandas.DataFrame: El DataFrame final, listo para el entrenamiento.
    """
    # --- 1. CONFIGURACIÓN INTERNA ---
    user = "itam_hackathon@hudl.com"
    password = "pGwIprel"
    creds = {"user": user, "passwd": password}

    all_events_dfs = []

    # --- Obtenemos la información de TODOS los partidos primero para eficiencia ---
    print("Descargando la información de todos los partidos de las temporadas especificadas...")
    all_matches_info = pd.DataFrame()
    season_ids_reales = [108, 235, 281, 317] # Los IDs de las temporadas que usaremos

    for season_id in season_ids_reales:
        try:
            season_matches = sb.matches(competition_id=73, season_id=season_id, creds=creds)
            all_matches_info = pd.concat([all_matches_info, season_matches])
        except Exception as e:
            print(f"No se pudo obtener la información de los partidos para la temporada {season_id}. Error: {e}")

    if all_matches_info.empty:
        print("No se pudo obtener la información de ningún partido. Terminando.")
        return None

    print("Información de partidos descargada con éxito. Iniciando la descarga de eventos...")

    # --- 2. Bucle para descargar y procesar todos los eventos usando tus arrays ---
    for i, season_match_ids in enumerate(season_match_id_arrays):
        print(f"\nProcesando Temporada {i+1}...")

        for match_id in tqdm(season_match_ids, desc=f"Partidos Temporada {i+1}"):
            try:
                # --- Lógica de procesamiento validada para un solo partido ---
                df_events = sb.events(match_id=match_id, creds=creds)
                if df_events.empty:
                    continue

                # Obtenemos el nombre del equipo local desde la info de partidos
                match_info = all_matches_info[all_matches_info['match_id'] == match_id]
                if match_info.empty:
                    continue
                home_team_name = match_info['home_team'].iloc[0]

                # Obtenemos el ID del equipo local buscando por nombre en los eventos
                home_team_id = df_events[df_events['team'] == home_team_name]['team_id'].iloc[0]
                df_events['home_team_id'] = home_team_id

                # (El resto del código de transformación es el que ya validamos)
                df = df_events.sort_values(by=['period', 'timestamp']).reset_index(drop=True)
                df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S.%f', errors='coerce')
                df['eventSec'] = df['timestamp'].dt.hour * 3600 + df['timestamp'].dt.minute * 60 + df['timestamp'].dt.second + df['timestamp'].dt.microsecond / 1e6
                event_map = {'Pass': 0, 'Ball Receipt': 0, 'Duel': 1, '50/50': 1,'Foul Committed': 2, 'Shot': 3, 'Dribble': 4, 'Carry': 5,'Ball Recovery': 6, 'Interception': 6, 'Block': 6, 'Clearance': 6, 'Goal Keeper': 7, 'Dispossessed': 8, 'Miscontrol': 8, 'Pressure': 9}
                df['EventTypeId'] = df['type'].map(event_map).fillna(10).astype(int)
                df = df.rename(columns={'period': 'Period', 'minute': 'Minute'})
                df['x_coord'] = df['location'].apply(lambda loc: loc[0] if isinstance(loc, list) and len(loc) > 0 else np.nan)
                df['y_coord'] = df['location'].apply(lambda loc: loc[1] if isinstance(loc, list) and len(loc) > 1 else np.nan)
                df['X'] = (df['x_coord'] / 120) * 100
                df['Y'] = (df['y_coord'] / 80) * 100
                df[['X', 'Y']] = df[['X', 'Y']].fillna(50)
                df['IsHomeTeam'] = (df['team_id'].fillna(0) == df['home_team_id']).astype(int)
                df['IsGoal'] = (df['shot_outcome'] == 'Goal').astype(int)
                def calculate_is_accurate(row):
                    if row['type'] == 'Pass' and pd.isna(row['pass_outcome']): return 1
                    if row['type'] == 'Shot' and row['shot_outcome'] in ['Goal', 'Saved', 'Post']: return 1
                    if row['type'] == 'Dribble' and row['dribble_outcome'] == 'Complete': return 1
                    if row['type'] == 'Duel' and row['duel_outcome'] in ['Won', 'Success In Play', 'Success Out']: return 1
                    return 0
                df['IsAccurate'] = df.apply(calculate_is_accurate, axis=1)
                df['HomeGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 1)
                df['AwayGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 0)
                df['HomeScore'] = df.groupby('match_id')['HomeGoal'].cumsum().shift(1).fillna(0)
                df['AwayScore'] = df.groupby('match_id')['AwayGoal'].cumsum().shift(1).fillna(0)
                df['TimeDelta'] = df.groupby('match_id')['eventSec'].diff().fillna(0)

                all_events_dfs.append(df)

            except Exception as e:
                print(f"  -> No se pudo procesar el match_id {match_id}. Error: {e}")

    if not all_events_dfs:
        print("No se pudieron procesar eventos. Terminando.")
        return None

    # --- 3. Consolidación Final ---
    print("\nConsolidando y limpiando el DataFrame final...")
    df_final_all = pd.concat(all_events_dfs, ignore_index=True)

    final_columns = ['match_id', 'eventSec', 'EventTypeId', 'Period', 'Minute', 'X', 'Y', 'IsHomeTeam', 'IsAccurate', 'IsGoal', 'HomeScore', 'AwayScore', 'TimeDelta']
    df_final_all = df_final_all[final_columns].copy().rename(columns={'match_id': 'matchId'})

    print(f"\nProcesamiento completado. Total de eventos: {len(df_final_all)}")
    df_final_all.to_csv(output_filename, index=False)
    print(f"DataFrame guardado exitosamente en '{output_filename}'")

    return df_final_all

# --- CÓMO EJECUTAR LA FUNCIÓN ---
user = "itam_hackathon@hudl.com"
password = "pGwIprel"
creds = {"user": user, "passwd": password}
print("Creando listas de match_id para cada temporada...")
array_season_2021_2022 = sb.matches(competition_id=73, season_id=108, creds=creds)['match_id'].to_list()
array_season_2022_2023 = sb.matches(competition_id=73, season_id=235, creds=creds)['match_id'].to_list()
array_season_2023_2024 = sb.matches(competition_id=73, season_id=281, creds=creds)['match_id'].to_list()
array_season_2024_2025 = sb.matches(competition_id=73, season_id=317, creds=creds)['match_id'].to_list()
print("Listas de match_id creadas.")

final_training_df = create_full_lem_dataset_from_arrays(
    array_season_2021_2022,
    array_season_2022_2023,
    array_season_2023_2024,
    array_season_2024_2025
)

if final_training_df is not None:
    print("\n--- Vista previa del DataFrame final ---")
    display(final_training_df.head())

Creando listas de match_id para cada temporada...
Listas de match_id creadas.
Descargando la información de todos los partidos de las temporadas especificadas...
Información de partidos descargada con éxito. Iniciando la descarga de eventos...

Procesando Temporada 1...


Partidos Temporada 1:   0%|          | 0/342 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
/tmp/ipython-input-1782280078.py:64: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_events['home_team_id'] = home_team_id
Partidos Temporada 1:   0%|          | 1/342 [00:02<12:11,  2.14s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning:


Procesando Temporada 2...


Partidos Temporada 2:   0%|          | 0/342 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Partidos Temporada 2:   0%|          | 1/342 [00:02<11:58,  2.11s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True


Procesando Temporada 3...


Partidos Temporada 3:   0%|          | 0/340 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Partidos Temporada 3:   0%|          | 1/340 [00:02<13:37,  2.41s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True


Procesando Temporada 4...


Partidos Temporada 4:   0%|          | 0/340 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
Partidos Temporada 4:   0%|          | 1/340 [00:02<16:54,  2.99s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True


Consolidando y limpiando el DataFrame final...


In [ ]:
import pandas as pd
import numpy as np
from statsbombpy import sb
from tqdm import tqdm
import os

def process_season_data(competition_id, season_id, creds, output_filename):
    """
    Descarga y procesa todos los datos de una temporada específica, aplicando un mapeo
    refinado y una limpieza final de los eventos de inicio de partido.
    """
    print(f"--- Iniciando procesamiento para la temporada ID: {season_id} ---")
    all_events_dfs = []

    try:
        matches = sb.matches(competition_id=competition_id, season_id=season_id, creds=creds)
        print(f"Se encontraron {len(matches)} partidos para la temporada {season_id}.")

        for index, match_info in tqdm(matches.iterrows(), total=matches.shape[0], desc=f"Procesando Partidos"):
            match_id = match_info['match_id']
            home_team_name = match_info['home_team']

            df_events = sb.events(match_id=match_id, creds=creds)
            if df_events.empty:
                continue

            # Eliminamos eventos de alineación que no son parte del juego
            df_events = df_events[df_events['type'] != 'Starting XI']

            home_team_id_series = df_events[df_events['team'] == home_team_name]['team_id']
            if home_team_id_series.empty:
                continue
            home_team_id = home_team_id_series.iloc[0]
            df_events['home_team_id'] = home_team_id

            # (Aplicamos toda la lógica de transformación)
            df = df_events.sort_values(by=['period', 'timestamp']).reset_index(drop=True)
            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%H:%M:%S.%f', errors='coerce')
            df['eventSec'] = df['timestamp'].dt.hour * 3600 + df['timestamp'].dt.minute * 60 + df['timestamp'].dt.second + df['timestamp'].dt.microsecond / 1e6

            # Mapeo refinado para separar Pase y Recepción
            event_map = {
                'Pass': 0, 'Ball Receipt': 1, 'Duel': 2, '50/50': 2,
                'Foul Committed': 3, 'Shot': 4, 'Dribble': 5, 'Carry': 6,
                'Ball Recovery': 7, 'Interception': 7, 'Block': 7, 'Clearance': 7,
                'Goal Keeper': 8, 'Dispossessed': 9, 'Miscontrol': 9, 'Pressure': 10,
                'Half Start': 11 # ID específico para el inicio del período
            }
            df['EventTypeId'] = df['type'].map(event_map).fillna(12).astype(int) # 12 para 'Otro'

            df = df.rename(columns={'period': 'Period', 'minute': 'Minute'})
            df['x_coord'] = df['location'].apply(lambda loc: loc[0] if isinstance(loc, list) and len(loc) > 0 else np.nan)
            df['y_coord'] = df['location'].apply(lambda loc: loc[1] if isinstance(loc, list) and len(loc) > 1 else np.nan)
            df['X'] = (df['x_coord'] / 120) * 100
            df['Y'] = (df['y_coord'] / 80) * 100
            df[['X', 'Y']] = df[['X', 'Y']].fillna(50)
            df['IsHomeTeam'] = (df['team_id'].fillna(0) == df['home_team_id']).astype(int)
            df['IsGoal'] = (df['shot_outcome'] == 'Goal').astype(int)
            def calculate_is_accurate(row):
                if row['type'] == 'Pass' and pd.isna(row['pass_outcome']): return 1
                if row['type'] == 'Shot' and row['shot_outcome'] in ['Goal', 'Saved', 'Post']: return 1
                if row['type'] == 'Dribble' and row['dribble_outcome'] == 'Complete': return 1
                if row['type'] == 'Duel' and row['duel_outcome'] in ['Won', 'Success In Play', 'Success Out']: return 1
                return 0
            df['IsAccurate'] = df.apply(calculate_is_accurate, axis=1)
            df['HomeGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 1)
            df['AwayGoal'] = (df['IsGoal'] == 1) & (df['IsHomeTeam'] == 0)
            df['HomeScore'] = df.groupby('match_id')['HomeGoal'].cumsum().shift(1).fillna(0)
            df['AwayScore'] = df.groupby('match_id')['AwayGoal'].cumsum().shift(1).fillna(0)
            df['TimeDelta'] = df.groupby('match_id')['eventSec'].diff().fillna(0)
            all_events_dfs.append(df)

        # Consolidación final para la temporada
        df_season = pd.concat(all_events_dfs, ignore_index=True)
        final_columns = ['match_id', 'eventSec', 'EventTypeId', 'Period', 'Minute', 'X', 'Y', 'IsHomeTeam', 'IsAccurate', 'IsGoal', 'HomeScore', 'AwayScore', 'TimeDelta']
        df_final = df_season[final_columns].copy().rename(columns={'match_id': 'matchId'})

        # --- NUEVO PASO DE LIMPIEZA FINAL ---
        print("\nAplicando limpieza final a los eventos de inicio de partido...")
        indices_to_drop = []
        for match_id in df_final['matchId'].unique():
            match_df = df_final[df_final['matchId'] == match_id]
            for period in match_df['Period'].unique():
                period_df = match_df[match_df['Period'] == period]
                # Encontrar el primer pase (ID 0) o recepción (ID 1)
                first_play_indices = period_df[(period_df['EventTypeId'] == 0) | (period_df['EventTypeId'] == 1)].index
                if not first_play_indices.empty:
                    first_play_index = first_play_indices[0]
                    # Seleccionar eventos ANTES del primer juego real
                    pre_game_events = period_df.loc[:first_play_index-1]
                    # Identificar los eventos de inicio (ID 11) que no son el último
                    start_events = pre_game_events[pre_game_events['EventTypeId'] == 11]
                    if len(start_events) > 1:
                        indices_to_drop.extend(start_events.index[:-1])

        df_final.drop(indices_to_drop, inplace=True)
        print(f"Limpieza finalizada. Se eliminaron {len(indices_to_drop)} eventos de inicio redundantes.")

        df_final.to_csv(output_filename, index=False)
        print(f"\nProcesamiento para la temporada {season_id} completado. Datos guardados en '{output_filename}'")
        return df_final.reset_index(drop=True)
    except Exception as e:
        print(f"\nOcurrió un error procesando la temporada {season_id}: {e}")
        return None

In [ ]:
# Configuración
user = "itam_hackathon@hudl.com"
password = "pGwIprel"
creds = {"user": user, "passwd": password}

# Procesar la primera temporada
df_2021_2022 = process_season_data(
    competition_id=73,
    season_id=108,
    creds=creds,
    output_filename="StatsBomb_LEM_2021-2022.csv"
)

if df_2021_2022 is not None:
    print("\nVista previa del DataFrame de la temporada 2021/2022:")
    display(df_2021_2022.head())

--- Iniciando procesamiento para la temporada ID: 108 ---
Se encontraron 342 partidos para la temporada 108.


Procesando Partidos:   0%|          | 0/342 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  events = pd.concat([*events.values()], axis=0, ignore_index=True, sort=True)
/tmp/ipython-input-251371692.py:34: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_events['home_team_id'] = home_team_id
Procesando Partidos:   0%|          | 1/342 [00:02<11:24,  2.01s/it]/usr/local/lib/python3.12/dist-packages/statsbombpy/sb.py:122: FutureWarning: Th


Aplicando limpieza final a los eventos de inicio de partido...
Limpieza finalizada. Se eliminaron 686 eventos de inicio redundantes.

Procesamiento para la temporada 108 completado. Datos guardados en 'StatsBomb_LEM_2021-2022.csv'

Vista previa del DataFrame de la temporada 2021/2022:


,matchId,eventSec,EventTypeId,Period,Minute,X,Y,IsHomeTeam,IsAccurate,IsGoal,HomeScore,AwayScore,TimeDelta
0,3816809,0.000,11,1,0,50.000000,50.000,0,0,0,0.0,0.0,0.000
1,3816809,1.968,0,1,0,50.000000,50.000,0,1,0,0.0,0.0,1.968
2,3816809,3.783,12,1,0,30.083333,50.125,0,0,0,0.0,0.0,1.815
3,3816809,3.783,6,1,0,30.083333,50.125,0,0,0,0.0,0.0,0.000
4,3816809,4.907,10,1,0,66.250000,50.000,1,0,0,0.0,0.0,1.124


In [ ]:
# Use value_counts() to count the occurrences of each unique value in 'EventTypeId'
event_type_counts = df_2021_2022['EventTypeId'].value_counts()

# Display the counts
print("Counts of each EventTypeId in df_2021_2022:")
print(event_type_counts)

Counts of each EventTypeId in df_2021_2022:
EventTypeId
12    283002
0     281669
6     218396
10     97220
7      64909
2      25671
9      17400
5      10554
8      10273
3       9640
4       8669
11       696
Name: count, dtype: int64


In [ ]:
# Create a shifted column to easily access the previous row's EventTypeId
df_2021_2022['prev_EventTypeId'] = df_2021_2022['EventTypeId'].shift(1)

# Identify the rows that meet the condition:
# Current EventTypeId is 12 AND Previous EventTypeId is 0
condition = (df_2021_2022['EventTypeId'] == 12) & (df_2021_2022['prev_EventTypeId'] == 0)

# Change the EventTypeId to 1 for the rows that meet the condition
df_2021_2022.loc[condition, 'EventTypeId'] = 1

# Drop the temporary 'prev_EventTypeId' column
df_2021_2022 = df_2021_2022.drop(columns=['prev_EventTypeId'])

# Display the updated counts to see the change
print("Counts of each EventTypeId after modification:")
print(df_2021_2022['EventTypeId'].value_counts())

# Display the head of the modified DataFrame to see the changes
print("\nPreview of the modified df_2021_2022:")
display(df_2021_2022.head())

# Guardamos es nuevo df
df_2021_2022.to_csv("StatsBomb_LEM_2021-2022_modificado.csv", index=False)

Counts of each EventTypeId after modification:
EventTypeId
0     281669
6     218396
1     215755
10     97220
12     67247
7      64909
2      25671
9      17400
5      10554
8      10273
3       9640
4       8669
11       696
Name: count, dtype: int64

Preview of the modified df_2021_2022:


,matchId,eventSec,EventTypeId,Period,Minute,X,Y,IsHomeTeam,IsAccurate,IsGoal,HomeScore,AwayScore,TimeDelta
0,3816809,0.000,11,1,0,50.000000,50.000,0,0,0,0.0,0.0,0.000
1,3816809,1.968,0,1,0,50.000000,50.000,0,1,0,0.0,0.0,1.968
2,3816809,3.783,1,1,0,30.083333,50.125,0,0,0,0.0,0.0,1.815
3,3816809,3.783,6,1,0,30.083333,50.125,0,0,0,0.0,0.0,0.000
4,3816809,4.907,10,1,0,66.250000,50.000,1,0,0,0.0,0.0,1.124


In [ ]:
import pandas as pd

# --- PASO 1: Carga tu DataFrame ya procesado ---
# Asegúrate de que el nombre del archivo CSV sea el correcto
try:
    df = df_2021_2022
    print("DataFrame cargado con éxito.")
    print("Conteo de EventTypeId ANTES de la corrección:")
    print(df['EventTypeId'].value_counts())

    # --- PASO 2: Crear columnas de ayuda para ver eventos pasados ---
    # Creamos columnas que nos dicen el tipo de evento de las dos filas anteriores
    df['PrevEventTypeId_1'] = df['EventTypeId'].shift(1) # Evento inmediatamente anterior
    df['PrevEventTypeId_2'] = df['EventTypeId'].shift(2) # Evento dos pasos atrás

    # También necesitamos saber quién hizo el pase original
    df['PasserIsHomeTeam'] = df['IsHomeTeam'].shift(2)

    # --- PASO 3: Definir las condiciones para la corrección ---

    # Condición 1: La secuencia de eventos es 0 -> 10 -> 12
    sequence_condition = (df['EventTypeId'] == 12) & \
                         (df['PrevEventTypeId_1'] == 10) & \
                         (df['PrevEventTypeId_2'] == 0)

    # Condición 2a: El equipo actual es el MISMO que el que pasó el balón -> es una Recepción (1)
    same_team_condition = sequence_condition & (df['IsHomeTeam'] == df['PasserIsHomeTeam'])

    # Condición 2b: El equipo actual es el OPONENTE del que pasó el balón -> es una Intercepción (7)
    opponent_team_condition = sequence_condition & (df['IsHomeTeam'] != df['PasserIsHomeTeam'])

    # --- PASO 4: Aplicar los cambios ---
    # Guardamos el número de cambios para verificar
    num_receipts_fixed = len(df[same_team_condition])
    num_interceptions_fixed = len(df[opponent_team_condition])

    # Aplicamos las correcciones usando .loc
    df.loc[same_team_condition, 'EventTypeId'] = 1
    df.loc[opponent_team_condition, 'EventTypeId'] = 7

    # --- PASO 5: Limpieza y Verificación ---
    # Eliminamos las columnas de ayuda que ya no necesitamos
    df.drop(columns=['PrevEventTypeId_1', 'PrevEventTypeId_2', 'PasserIsHomeTeam'], inplace=True)

    print("\n--- Corrección Avanzada Aplicada ---")
    print(f"Se corrigieron {num_receipts_fixed} eventos a 'Recepción de Balón' (1).")
    print(f"Se corrigieron {num_interceptions_fixed} eventos a 'Acción Defensiva' (7).")

    print("\nConteo de EventTypeId DESPUÉS de la corrección:")
    print(df['EventTypeId'].value_counts())

    # Opcional: Guardar el nuevo DataFrame super-corregido
    df.to_csv("StatsBomb_LEM_2021-2022_super_corregido.csv", index=False)
    print("\nDataFrame super-corregido guardado con éxito.")

except FileNotFoundError:
    print("Error: No se encontró el archivo CSV. Asegúrate de que el nombre y la ruta sean correctos.")

DataFrame cargado con éxito.
Conteo de EventTypeId ANTES de la corrección:
EventTypeId
0     281669
6     218396
1     215755
10     97220
12     67247
7      64909
2      25671
9      17400
5      10554
8      10273
3       9640
4       8669
11       696
Name: count, dtype: int64

--- Corrección Avanzada Aplicada ---
Se corrigieron 19252 eventos a 'Recepción de Balón' (1).
Se corrigieron 129 eventos a 'Acción Defensiva' (7).

Conteo de EventTypeId DESPUÉS de la corrección:
EventTypeId
0     281669
1     235007
6     218396
10     97220
7      65038
12     47866
2      25671
9      17400
5      10554
8      10273
3       9640
4       8669
11       696
Name: count, dtype: int64

DataFrame super-corregido guardado con éxito.
